In [5]:
import sys
from pathlib import Path
import pandas as pd
import os

# Configuration des chemins
ROOT_DIR = Path("..").resolve()
sys.path.append(str(ROOT_DIR))

# Autoreload
%load_ext autoreload
%autoreload 2

from src.asr_inference import ASRPredictor

print(f"Racine du projet : {ROOT_DIR}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Racine du projet : C:\Users\benic\Documents\Projet_DL_Translation


In [6]:
# On charge le modèle "small" (environ 500Mo de RAM, assez rapide)
# Tu pourras tester "base" (plus rapide) ou "medium" (plus précis mais lent)
predictor = ASRPredictor(root_dir=ROOT_DIR, model_size="medium")

--- Initialisation ASR ---
Device detecte : CPU
Chargement du modele Whisper (medium). Cela peut prendre un instant...


100%|█████████████████████████████████████| 1.42G/1.42G [02:17<00:00, 11.1MiB/s]


✅ Modele charge et pret !


In [7]:
print("=== ÉTAPE 1 : Test sur une vidéo complète (Génération SRT) ===")

# On récupère le premier ID de vidéo du split VALID
valid_csv = ROOT_DIR / "data" / "processed" / "valid_asr_fr.csv"
df_valid = pd.read_csv(valid_csv)
first_video_id = df_valid['video_id'].iloc[0]

archive_name = "mtedx_fr.tgz"
srt_output_dir = ROOT_DIR / "data" / "processed" / "subtitles"
srt_output_dir.mkdir(parents=True, exist_ok=True)

try:
    # 1. On extrait le FLAC complet temporairement
    flac_path = predictor.extract_single_flac(archive_name, first_video_id)
    
    # 2. On lance Whisper sur la vidéo entière pour créer le SRT
    srt_path = srt_output_dir / f"{first_video_id}.srt"
    predictor.transcribe_full_video_to_srt(flac_path, srt_path, language="fr")
    
    # 3. Nettoyage
    os.remove(flac_path)
    
    # Aperçu des premières lignes du SRT généré !
    print("\n--- Aperçu du fichier SRT généré ---")
    with open(srt_path, "r", encoding="utf-8") as f:
        print("".join([next(f) for _ in range(20)])) # Affiche les 10 premières lignes
        
except Exception as e:
    print(f"Erreur : {e}")

=== ÉTAPE 1 : Test sur une vidéo complète (Génération SRT) ===
Extraction temporaire de 9fxo9YJhnG8.flac...
Transcription de la video complete : 9fxo9YJhnG8.flac


c:\Users\benic\Documents\Projet_DL_Translation\venv_dl\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ Fichier SRT genere avec succes : C:\Users\benic\Documents\Projet_DL_Translation\data\processed\subtitles\9fxo9YJhnG8.srt

--- Aperçu du fichier SRT généré ---
1
00:00:00,000 --> 00:00:00,660
Derniers

2
00:00:00,660 --> 00:00:02,000
gossip

3
00:00:04,480 --> 00:00:05,900
Janxième

4
00:00:06,100 --> 00:00:07,580
Vers егоungen

5
00:00:07,600 --> 00:00:09,620
Volumee




In [ ]:
print("=== ÉTAPE 2 : Inférence sur les segments (Pour évaluation) ===")

# On va transcrire un échantillon de 20 segments pour valider le concept (et ne pas attendre 3 heures)
# Mets limit=None quand tu voudras faire tout le dataset valid
LIMIT_TEST = None 

df_results = predictor.transcribe_segments_to_csv(
    csv_path=valid_csv,
    split="valid",
    limit=LIMIT_TEST
)

# On affiche la grande table demandée pour le comparatif visuel
print("\n--- Comparatif Visuel (Cible vs Prédit) ---")
display_cols = ['segment_id', 'start_time', 'end_time', 'text_fr', 'text_pred']

# Configuration Pandas pour bien voir le texte sans qu'il soit coupé
pd.set_option('display.max_colwidth', None)
display(df_results[display_cols].head(10))
pd.reset_option('display.max_colwidth')

=== ÉTAPE 2 : Inférence sur les segments (Pour évaluation) ===
--- Transcription des segments (VALID) ---


Inference ASR:  81%|████████  | 835/1036 [3:35:27<47:47, 14.26s/it]   